In [18]:
import numpy as np
import pandas as pd
import itertools

In [19]:
class Wavefunction(object):
    """a wavefunction representing a quantum state"""
    
    def __init__(self, states, amplitude_vector):
        
        if len(amplitude_vector) == 0 or len(amplitude_vector) & (len(amplitude_vector) - 1) != 0:
            raise TypeError("Amplitude vector must have a length that is a power of two")
        self.wave = pd.DataFrame([amplitude_vector], columns = states)
        sumprob = np.sum(self.probabilities())
        if not np.isclose(sumprob, 1.0):
            raise ValueError("The wavefunction is not normalized. "
                             "The probabilities sum to {} instead of 1".format(sumprob))
    
    def probabilities(self):
        """returns a dictionary of associated probabilities."""
        return np.abs(self.amplitude()) ** 2
    
    def print_state(self):
        """represent a quantum state in bra-ket notations"""
        states = self.wave.columns
        string = str(self.wave.loc[0, states[0]]) + '|' + states[0] + '>'
        for state in states[1:]:
            string += ' + ' + str(self.wave.loc[0, state]) + '|' + state + '>'
        return string
    
    def measure_sample(self, n_samples):
        """Make a measurement on quibits"""
        inds = np.random.choice(len(self.wave.columns), n_samples, p=self.probabilities())
        return np.unique(np.array(self.wave.columns[inds]), return_counts=True)
    
    def state(self):
        """return an array of possibly states"""
        return np.array(self.wave.columns)
    
    def amplitude(self):
        """return an array of associated amplitude-vectors"""
        return np.array(self.wave.iloc[0, :])

In [20]:
def Qubit(qubit_num):
    """create a quantum circuit"""
    states = list(itertools.product(('0', '1'), repeat=qubit_num))
    for i in range(len(states)):
        states[i] = ''.join(states[i])  
    amplitude_vector = np.zeros(2**qubit_num, dtype = complex)
    amplitude_vector[0] = 1.0
    return Wavefunction(states, amplitude_vector)

In [21]:
def H(wavefunction, n):
    """Hadamard gate: math:`\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}`"""
    states = wavefunction.state()
    amplitude = wavefunction.amplitude()
    qubit_num = len(states[0])
    new_amplitude = np.zeros(2**qubit_num, dtype = complex)
    cut = 2**(qubit_num-n-1)
    if n >= qubit_num or n < 0:
        raise TypeError("Index is out of range")
    for i in range(2**qubit_num):
        if states[i][n] == '0':
            new_amplitude[i] += amplitude[i]/2**0.5
            new_amplitude[i+cut] += amplitude[i]/2**0.5
        else:
            new_amplitude[i] -= amplitude[i]/2**0.5
            new_amplitude[i-cut] += amplitude[i]/2**0.5  
    wavefunction.wave.iloc[0, :] = new_amplitude
#     return Wavefunction(states, new_amplitude)
                       

In [22]:
def X(wavefunction, n):
    """Flip gate: math:`\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}`"""
    states = wavefunction.state()
    amplitude = wavefunction.amplitude()
    qubit_num = len(states[0])
    new_amplitude = np.zeros(2**qubit_num, dtype = complex)
    cut = 2**(qubit_num-n-1)
    if n >= qubit_num or n < 0:
        raise TypeError("Index is out of range")
    for i in range(2**qubit_num):
        if states[i][n] == '0':
            new_amplitude[i+cut] += amplitude[i]
        else:
            new_amplitude[i-cut] = amplitude[i]  
    wavefunction.wave.iloc[0, :] = new_amplitude
#     return Wavefunction(states, new_amplitude)

In [25]:
n = 1
circle = Qubit(n)

circle.measure_sample(10)

(array(['0'], dtype=object), array([10]))

In [26]:
H(circle,1)
circle.print_state()

TypeError: Index is out of range

In [27]:
n = 5
circle = Qubit(n)

H(circle,0)
H(circle,1)
H(circle,2)
circle.print_state()

'(0.3535533905932737+0j)|00000> + 0j|00001> + 0j|00010> + 0j|00011> + (0.3535533905932737+0j)|00100> + 0j|00101> + 0j|00110> + 0j|00111> + (0.3535533905932737+0j)|01000> + 0j|01001> + 0j|01010> + 0j|01011> + (0.3535533905932737+0j)|01100> + 0j|01101> + 0j|01110> + 0j|01111> + (0.3535533905932737+0j)|10000> + 0j|10001> + 0j|10010> + 0j|10011> + (0.3535533905932737+0j)|10100> + 0j|10101> + 0j|10110> + 0j|10111> + (0.3535533905932737+0j)|11000> + 0j|11001> + 0j|11010> + 0j|11011> + (0.3535533905932737+0j)|11100> + 0j|11101> + 0j|11110> + 0j|11111>'

In [28]:
circle.measure_sample(100000)

(array(['00000', '00100', '01000', '01100', '10000', '10100', '11000',
        '11100'], dtype=object),
 array([12593, 12624, 12351, 12543, 12365, 12603, 12414, 12507]))

In [29]:
n = 10
circle = Qubit(n)

H(circle,0)
H(circle,1)
H(circle,2)
H(circle,2)
H(circle,0)
H(circle,1)
circle.print_state()

'(0.9999999999999996+0j)|0000000000> + 0j|0000000001> + 0j|0000000010> + 0j|0000000011> + 0j|0000000100> + 0j|0000000101> + 0j|0000000110> + 0j|0000000111> + 0j|0000001000> + 0j|0000001001> + 0j|0000001010> + 0j|0000001011> + 0j|0000001100> + 0j|0000001101> + 0j|0000001110> + 0j|0000001111> + 0j|0000010000> + 0j|0000010001> + 0j|0000010010> + 0j|0000010011> + 0j|0000010100> + 0j|0000010101> + 0j|0000010110> + 0j|0000010111> + 0j|0000011000> + 0j|0000011001> + 0j|0000011010> + 0j|0000011011> + 0j|0000011100> + 0j|0000011101> + 0j|0000011110> + 0j|0000011111> + 0j|0000100000> + 0j|0000100001> + 0j|0000100010> + 0j|0000100011> + 0j|0000100100> + 0j|0000100101> + 0j|0000100110> + 0j|0000100111> + 0j|0000101000> + 0j|0000101001> + 0j|0000101010> + 0j|0000101011> + 0j|0000101100> + 0j|0000101101> + 0j|0000101110> + 0j|0000101111> + 0j|0000110000> + 0j|0000110001> + 0j|0000110010> + 0j|0000110011> + 0j|0000110100> + 0j|0000110101> + 0j|0000110110> + 0j|0000110111> + 0j|0000111000> + 0j|000011

In [30]:
circle.measure_sample(100000)

(array(['0000000000'], dtype=object), array([100000]))

In [ ]:
n = 10000
circle = Qubit(n)
circle.print_state()

In [ ]:
X(circle, 2)
circle.print_state()

In [ ]:
n = 100
circle = Qubit(n)
X(circle, 0)
X(circle, 1)
X(circle, 2)
circle.print_state()

In [ ]:
n = 10
circle = Qubit(n)
X(circle, 0)
H(circle, 0)
circle.print_state()

In [16]:
circle.measure_sample(100000)

(array(['0', '1'], dtype=object), array([49838, 50162]))